# 4.5 读取和存储

In [1]:
import torch
from torch import nn
import os
import sys

In [2]:
# 演示Tensor的读写

if sys.platform.startswith('win'):
    # 如果是windows
    directory = os.path.join(os.getcwd(), 'Datasets', '14_TempSaveData')
else:
    # linux
    directory = os.path.join(os.getcwd(), '..', 'Datasets', '14_TempSaveData')

if not os.path.exists(directory):
    os.makedirs(directory)

x = torch.ones(3)
torch.save(x, os.path.join(directory, 'x.pt'))

x2 = torch.load(os.path.join(directory, 'x.pt'))
x2

y = torch.zeros(4)
torch.save([x, y], os.path.join(directory, 'xy.pt'))
xy_list = torch.load(os.path.join(directory, 'xy.pt'))
xy_list

torch.save({'x': x, 'y': y}, os.path.join(directory, 'xy_dict.pt'))
xy = torch.load(os.path.join(directory, 'xy_dict.pt'))
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

state_dict()的用法，它是用来记录网络中的关键字的

In [3]:
# 注意，只有具有可学习参数的层(卷积层、线性层等)才有state_dict中的条目。
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)


net = MLP()
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.3278, -0.1675,  0.0768],
                      [ 0.1002, -0.5686, -0.4736]])),
             ('hidden.bias', tensor([-0.0877,  0.3716])),
             ('output.weight', tensor([[0.3007, 0.3152]])),
             ('output.bias', tensor([0.3875]))])

优化器(optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息。

In [4]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

### 如何保存网络

```
保存方式1（仅参数）
torch.save(model.state_dict(), PATH)  # 推荐的文件后缀名是pt或pth
加载方式1
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))

保存方式2（整个模型）
torch.save(model, PATH)
加载方式2
model = torch.load(PATH)
```

In [5]:
# 第一种实践
X = torch.randn(2, 3)
Y = net(X)

PATH = os.path.join(directory, 'net.pt')
torch.save(net.state_dict(), PATH)

net2 = MLP()
net2.load_state_dict(torch.load(PATH))
Y2 = net2(X)
print(Y2 == Y)

tensor([[True],
        [True]])
